# Mock data generation

In [5]:
import pandas as pd
import numpy as np
import random

In [7]:
df = pd.read_excel('mock_random.xlsx')

# 1. Generate a random dataset (without error)

## Setting

- 2 years (2017, 2018)
- 3 grades each year (1, 2, 3)
- 1000 students each grade
- Each student ranks 5 schools
- 25 schools
- Capacity at each school: random between 50 - 60
- 2 tie-breaker student groups (0, 1)
- 3 types of advantage (0.1, 0.3, 0.5) applied to student group 1
- Priority: random among (0,1,2,3,4,5)
- Priority = 0: guaranteed assignment
- Outcome 1,2, Covariate 1,2: student-specific
- Covariate 3: stduent-school specific (e.g. distance)

In [588]:
# Define the parameters

years = [2017, 2018]
grades = [1,2,3]
num_students_per_grade = 1000
num_schools = 25
max_num_schools_per_student = 5
num_tie_breakers = 20

In [589]:
# studnet-specific information

num_total_students = len(years) * len(grades) * num_students_per_grade
all_students = list(range(1, num_total_students + 1))
random.shuffle(all_students)

# default tie-breaker value
tie_dict = {}
for n in range(1, num_total_students+1):
    tie_dict[n] = {}
    for t in range(1, num_tie_breakers+1):
        tie_dict[n][t] = random.uniform(0, 1)
            
# outcomes 1, 2
outcome_dict = {}
for n in range(1, num_total_students+1):
    outcome_dict[n] = {}
    outcome_dict[n] = (random.randint(0, 100), random.randint(0, 100))

# covariate 1, 2
covariate_dict = {}
for n in range(1, num_total_students+1):
    covariate_dict[n] = {}
    covariate_dict[n] = (random.randint(0, 100), random.randint(0, 100))

In [590]:
# (school, year, grade) - speicific information

#capacity
capacity_dict = {}
for n in range(1, num_schools+1):
    capacity_dict[n] = {}
    for y in years:
        capacity_dict[n][y] = {}
        for g in grades:
            capacity_dict[n][y][g] = random.randint(50,60)

# treatment
treatment_dict = {}
for n in range(1, num_schools+1):
    treatment_dict[n] = {}
    for y in years:
        treatment_dict[n][y] = {}
        for g in grades:
            treatment_dict[n][y][g] = random.randint(0, 1)
            
# advantage
advantage_dict = {}
for n in range(1, num_schools+1):
    advantage_dict[n] = {}
    for y in years:
        advantage_dict[n][y] = {}
        for g in grades:
            advantage_dict[n][y][g] = random.choice([0.1, 0.3, 0.5])

In [591]:
# student - school pair specific information

# covariate 3 (e.g., distance)
covariate3_dict = {}
for n in range(1, num_total_students+1):
    covariate3_dict[n] = {}
    for s in range(1, num_schools+1):
        covariate3_dict[n][s] = random.randint(10, 500)

In [592]:
# Define the column names
columns = ['Student ID', 'Year', 'Grade', 'Choice Rank', 'School ID', 'Treatment', 'Capacity', 
           'Priority', 'Default Tie-breaker Index', 'Tie-breaker Student Group Index', 
           'Advantage', 'Default Tie-breaker', 'Effective Tie-breaker', 'Assignment', 'Enrollment', 
           'Outcome 1', 'Outcome 2', 'Covariate 1', 'Covariate 2', 'Covariate 3']

In [593]:
# Create an empty list to store the data
data = []

In [594]:
# Loop through the years, grades, and schools to generate the data
count = 0
for year in years:
    for grade in grades:
        students = all_students[count*1000: (count+1)*1000]
        count += 1
        for student_index in students:
            student_id = student_index
            # Shuffle the list of school IDs to randomize the order
            school_ids = list(range(1, num_schools+1))
            random.shuffle(school_ids)
            # Take the first max_num_schools_per_student schools for this student
            school_ids = school_ids[:max_num_schools_per_student]
            
            # assignment 
            assigned_school = random.choice(school_ids)
            
            # enrollment
            if random.uniform(0,1) < 0.8:
                enrolled_school = assigned_school
            else:
                enrolled_school = random.choice(school_ids)

            # Loop through the school IDs and assign them to this student
            for rank, school_id in enumerate(school_ids):
                capacity = capacity_dict[school_id][year][grade] # Random capacity between 500 and 1000
                treatment = treatment_dict[school_id][year][grade] # Random treatment
                
                priority = random.randint(1, 5) # Random priority ranking 
                if random.uniform(0,1) < 0.05:
                    priority = 0
                
                default_tie_breaker_index = random.randint(1, num_tie_breakers) # Random default tie-breaker index
                tie_breaker_student_group_index = random.randint(0, 1) # Random tie-breaker student group index
                
                if tie_breaker_student_group_index == 0:
                    advantage = 0
                else:
                    advantage = advantage_dict[school_id][year][grade]
                
                default_tie_breaker = tie_dict[student_id][default_tie_breaker_index]
                effective_tie_breaker = default_tie_breaker * (1 - advantage)
                
                if school_id == assigned_school:
                    assignment = 1
                else:
                    assignment = 0
                
                if school_id == enrolled_school:
                    enrollment = 1
                else:
                    enrollment = 0
               
                outcome_1 = outcome_dict[student_id][0]
                outcome_2 = outcome_dict[student_id][1]
                covariate_1 = covariate_dict[student_id][0]
                covariate_2 = covariate_dict[student_id][1]
                covariate_3 = covariate3_dict[student_id][school_id]
                
                # Create a list of values for this row
                row = [student_id, year, grade, rank+1, school_id, treatment, capacity, 
                       priority, default_tie_breaker_index, tie_breaker_student_group_index, 
                       advantage, default_tie_breaker, effective_tie_breaker, assignment, 
                       enrollment, outcome_1, outcome_2, covariate_1, covariate_2, covariate_3]
                data.append(row)

In [619]:
df = pd.DataFrame(data, columns = columns)
df = df.sort_values(['Student ID', 'Choice Rank'])
df = df.reset_index(drop=True)
df.head()

,Student ID,Year,Grade,Choice Rank,School ID,Treatment,Capacity,Priority,Default Tie-breaker Index,Tie-breaker Student Group Index,Advantage,Default Tie-breaker,Effective Tie-breaker,Assignment,Enrollment,Outcome 1,Outcome 2,Covariate 1,Covariate 2,Covariate 3
0,1,2018,2,1,14,1,52,1,13,1,0.1,0.716942,0.645247,1,1,38,2,34,96,17
1,1,2018,2,2,13,1,50,2,20,0,0.0,0.090022,0.090022,0,0,38,2,34,96,316
2,1,2018,2,3,6,1,57,3,4,0,0.0,0.994843,0.994843,0,0,38,2,34,96,101
3,1,2018,2,4,23,1,51,4,6,0,0.0,0.481525,0.481525,0,0,38,2,34,96,159
4,1,2018,2,5,12,0,56,1,4,0,0.0,0.994843,0.994843,0,0,38,2,34,96,183


In [596]:
# Guaranteed assignment
for i in range(1, num_total_students+1):
    df_new = df.loc[df['Student ID'] == i]
    for j in df_new.index:
        if df.loc[j, 'Priority'] == 0:
            df.loc[j, 'Assignment'] = 1
            a = list(df_new.index)
            a.remove(j)
            for k in a:
                df.loc[k, 'Assignment'] = 0

In [597]:
# Capacity
over_assign = []
over_enroll = []
for n in range(1, num_schools+1):
    for y in years:
        for g in grades:
            if df.loc[(df['School ID'] == n) & (df['Year'] == y) & (df['Grade'] == g)]['Assignment'].sum() > capacity_dict[n][y][g]:
                over_assign.append((n,y,g))
            if df.loc[(df['School ID'] == n) & (df['Year'] == y) & (df['Grade'] == g)]['Enrollment'].sum() > capacity_dict[n][y][g]:
                over_enroll.append((n,y,g))

In [598]:
over_assign

[(6, 2018, 3), (15, 2018, 2), (16, 2018, 1)]

In [599]:
over_enroll

[(16, 2018, 1), (20, 2017, 3)]

In [600]:
df.to_excel('random.xlsx')

## Check if it's (partly) clean

In [397]:
df.loc[(df['Priority'] == 0)]

,Student ID,Year,Grade,Choice Rank,School ID,Treatment,Capacity,Priority,Default Tie-breaker Index,Tie-breaker Student Group Index,Advantage,Default Tie-breaker,Effective Tie-breaker,Assignment,Enrollment,Outcome 1,Outcome 2,Covariate 1,Covariate 2,Covariate 3
36,8,2018,2,2,1,1,54,0,6,0,0.0,0.963809,0.963809,1,0,65,91,98,18,261
51,11,2018,3,2,6,0,51,0,3,1,0.5,0.167029,0.083514,1,0,57,81,35,52,298
59,12,2018,3,5,17,1,55,0,19,0,0.0,0.063748,0.063748,0,0,19,24,34,89,449
72,15,2017,1,3,6,0,48,0,20,0,0.0,0.447438,0.447438,0,0,56,92,63,14,24
77,16,2018,3,3,9,0,51,0,20,0,0.0,0.133346,0.133346,0,0,21,6,50,45,146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29926,5986,2018,2,2,18,0,53,0,6,0,0.0,0.086855,0.086855,1,0,93,32,9,72,365
29935,5988,2018,3,1,13,1,54,0,17,0,0.0,0.001799,0.001799,1,0,56,71,2,33,100
29959,5992,2017,3,5,6,0,51,0,7,1,0.3,0.380747,0.266523,0,0,62,67,43,78,102
29995,6000,2017,3,1,6,0,51,0,1,1,0.3,0.337636,0.236345,1,1,65,100,42,4,178


In [224]:
df.loc[(df['School ID'] == 15) & (df['Year'] == 2017)]

,Student ID,Year,Grade,Choice Rank,School ID,Treatment,Capacity,Priority,Default Tie-breaker Index,Tie-breaker Student Group Index,Advantage,Default Tie-breaker,Effective Tie-breaker,Assignment,Enrollment,Outcome 1,Outcome 2,Covariate 1,Covariate 2,Covariate 3
8,2,2017,3,4,15,0,53,4,18,1,0.1,0.686058,0.617453,0,0,99,61,74,76,359
125,26,2017,1,1,15,1,54,6,15,1,0.5,0.464505,0.232252,1,1,51,87,82,19,111
248,50,2017,3,4,15,0,53,9,14,1,0.1,0.505619,0.455057,1,1,45,65,85,37,155
330,67,2017,1,1,15,1,54,9,13,1,0.5,0.694898,0.347449,0,0,98,20,23,36,321
368,74,2017,2,4,15,1,51,3,3,0,0.0,0.870588,0.870588,0,0,91,88,66,27,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29768,5954,2017,1,4,15,1,54,4,4,0,0.0,0.582798,0.582798,0,0,48,29,28,52,192
29844,5969,2017,3,5,15,0,53,5,20,1,0.1,0.773818,0.696436,0,0,72,49,16,72,314
29912,5983,2017,2,3,15,1,51,2,6,0,0.0,0.591010,0.591010,0,0,67,58,46,50,421
29955,5992,2017,1,1,15,1,54,8,6,1,0.5,0.042254,0.021127,0,0,99,21,79,92,268


In [147]:
df.loc[(df['Student ID'] == 1502) & (df['Year'] == 2017)]

,Student ID,Year,Grade,Choice Rank,School ID,Treatment,Capacity,Priority,Default Tie-breaker Index,Tie-breaker Student Group Index,Advantage,Default Tie-breaker,Effective Tie-breaker,Assignment,Enrollment,Outcome 1,Outcome 2,Covariate 1,Covariate 2,Covariate 3
7505,1502,2017,3,1,19,0,657,7,15,1,0.3,0.065687,0.045981,0,0,16,90,69,2,50
7506,1502,2017,3,2,21,0,990,6,19,0,0.0,0.137443,0.137443,1,0,16,90,69,2,144
7507,1502,2017,3,3,25,0,954,10,19,1,0.5,0.460700,0.230350,0,0,16,90,69,2,483
7508,1502,2017,3,4,3,1,602,2,11,1,0.3,0.787812,0.551468,0,0,16,90,69,2,247
7509,1502,2017,3,5,24,1,887,8,16,1,0.1,0.311524,0.280371,0,1,16,90,69,2,31


# 2. Create issues that result in errors / warnings

## (erroneous cells are marked  red in the excel file)

## (1) Inconsistency within a student

### (1.1) Inconsistent grade within a student

Student ID 1 \
Grade \
index 2

### (1.2) Inconsistent Outcomes within a student

Student ID 11 \
Outcome 1 \
index 50

### (1.3) Inconsistent Default Tie-breaker within a (Student ID, Default Tie-breaker Index) pair

In [607]:
df.loc[(df['Student ID'] == 8)]

,Student ID,Year,Grade,Choice Rank,School ID,Treatment,Capacity,Priority,Default Tie-breaker Index,Tie-breaker Student Group Index,Advantage,Default Tie-breaker,Effective Tie-breaker,Assignment,Enrollment,Outcome 1,Outcome 2,Covariate 1,Covariate 2,Covariate 3
35,8,2017,1,1,22,1,54,1,8,0,0.0,0.598726,0.598726,0,0,52,23,9,6,139
36,8,2017,1,2,13,1,59,1,19,0,0.0,0.537666,0.537666,0,0,52,23,9,6,223
37,8,2017,1,3,11,0,59,1,20,0,0.0,0.899058,0.899058,0,0,52,23,9,6,493
38,8,2017,1,4,18,1,60,3,8,1,0.1,0.598726,0.538854,1,1,52,23,9,6,220
39,8,2017,1,5,8,0,58,2,16,1,0.3,0.853363,0.597354,0,0,52,23,9,6,373


Student ID 8 \
Default Tie-breaker \
index 38 and a lot more due to checkpoint #12 (correlation, orange mark in excel)

## (2) Inconsistency within a (school, year, grade)

### (2.1) Inconsistent Treatment value within a (School, Year, Grade)

In [605]:
df.loc[(df['School ID'] == 4) & (df['Year'] == 2017) & (df['Grade'] == 3)]

,Student ID,Year,Grade,Choice Rank,School ID,Treatment,Capacity,Priority,Default Tie-breaker Index,Tie-breaker Student Group Index,Advantage,Default Tie-breaker,Effective Tie-breaker,Assignment,Enrollment,Outcome 1,Outcome 2,Covariate 1,Covariate 2,Covariate 3
25,6,2017,3,1,4,1,51,2,11,1,0.3,0.867018,0.606913,1,1,80,43,97,57,279
60,13,2017,3,1,4,1,51,0,18,0,0.0,0.462623,0.462623,1,0,70,9,11,55,152
96,20,2017,3,2,4,1,51,1,20,0,0.0,0.681404,0.681404,0,0,62,58,0,52,251
115,24,2017,3,1,4,1,51,3,7,0,0.0,0.401073,0.401073,0,0,96,69,12,85,287
572,115,2017,3,3,4,1,51,4,13,0,0.0,0.498610,0.498610,0,0,25,27,78,29,416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29587,5918,2017,3,3,4,1,51,2,12,0,0.0,0.529794,0.529794,0,0,74,30,44,79,115
29701,5941,2017,3,2,4,1,51,3,5,0,0.0,0.114234,0.114234,0,0,80,85,3,3,167
29735,5948,2017,3,1,4,1,51,4,1,1,0.3,0.074436,0.052105,0,0,17,59,17,91,63
29979,5996,2017,3,5,4,1,51,4,17,1,0.3,0.787875,0.551513,0,0,61,7,10,20,344


School ID 4, 25 \
Treatment \
index 13, 25

### (2.2) Inconsistent Capacity within a (School, Year, Grade)

In [606]:
df.loc[(df['School ID'] == 7) & (df['Year'] == 2018) & (df['Grade'] == 2)]

,Student ID,Year,Grade,Choice Rank,School ID,Treatment,Capacity,Priority,Default Tie-breaker Index,Tie-breaker Student Group Index,Advantage,Default Tie-breaker,Effective Tie-breaker,Assignment,Enrollment,Outcome 1,Outcome 2,Covariate 1,Covariate 2,Covariate 3
31,7,2018,2,2,7,1,50,2,13,1,0.3,0.528189,0.369732,0,0,88,19,42,83,80
274,55,2018,2,5,7,1,50,5,15,0,0.0,0.160429,0.160429,0,0,6,31,38,95,144
667,134,2018,2,3,7,1,50,4,13,0,0.0,0.910433,0.910433,0,1,27,45,36,29,133
817,164,2018,2,3,7,1,50,5,8,1,0.3,0.021589,0.015113,0,0,97,43,9,23,374
1361,273,2018,2,2,7,1,50,3,10,1,0.3,0.832606,0.582824,0,1,54,47,44,5,230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29389,5878,2018,2,5,7,1,50,4,14,0,0.0,0.045618,0.045618,0,0,32,96,71,70,266
29593,5919,2018,2,4,7,1,50,3,14,0,0.0,0.739529,0.739529,0,0,36,21,12,9,298
29813,5963,2018,2,4,7,1,50,0,18,1,0.3,0.136353,0.095447,1,1,26,62,62,63,223
29901,5981,2018,2,2,7,1,50,1,5,0,0.0,0.056860,0.056860,0,0,77,40,54,40,284


School ID 7 \
Capacity \
index 31

### (2.3) Inconsistent advantage within a (School, Year, Grade) pair

In [8]:
df.loc[(df['School ID'] == 16) & (df['Year'] == 2017) & (df['Grade'] == 1)]

,Unnamed: 0,Student ID,Year,Grade,Choice Rank,School ID,Treatment,Capacity,Priority,Default Tie-breaker Index,...,Advantage,Default Tie-breaker,Effective Tie-breaker,Assignment,Enrollment,Outcome 1,Outcome 2,Covariate 1,Covariate 2,Covariate 3
40,40,9,2017,1,1,16,1,54,3,14,...,0.1,0.530556,0.477500,0,0,70,78,76,47,311
235,235,48,2017,1,1,16,1,54,2,6,...,0.0,0.480062,0.480062,0,0,24,42,84,30,113
406,406,82,2017,1,2,16,1,54,1,13,...,0.0,0.381522,0.381522,1,0,14,84,86,13,278
648,648,130,2017,1,4,16,1,54,1,9,...,0.1,0.665088,0.598579,0,0,84,48,99,32,488
881,881,177,2017,1,2,16,1,54,3,4,...,0.0,0.228207,0.228207,0,0,30,18,70,94,317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28872,28872,5775,2017,1,3,16,1,54,1,20,...,0.0,0.740661,0.740661,0,0,68,66,27,17,76
28971,28971,5795,2017,1,2,16,1,54,4,19,...,0.1,0.854284,0.768855,1,1,84,62,94,41,437
29001,29001,5801,2017,1,2,16,1,54,4,20,...,0.1,0.454356,0.408921,0,0,2,3,85,70,94
29800,29800,5961,2017,1,1,16,1,54,2,17,...,0.0,0.849780,0.849780,0,0,57,98,65,52,305


School ID 16 \
Advantage \
index 40

### (3) A student chose multiple schools for the same rank

Student ID 2, 3 \
Choice Rank \
index 6, 7, 12, 13

### (4) A student chose the same school for multiple ranks

Student ID 3, 4 \
School ID \
index 12, 13, 15, 16

### (5) Inconsecutive Choice Rank (e.g., 1,2,4)

Student ID 5 \
Choice Rank \
index 22, 23, 24

## (6) Multiple Assignment / Enrollment

### (6.1) A student is assigned to multiple schools

Student ID 12 \
Assignment \
index 55, 56

### (6.2) A student is enrolled in multiple schools

Student ID 13 \
Enrollment \
index 60, 61

## (7) Over capacity 

### (7.1) A (school, year, grade) is assigned with more students than its capacity and contains at least one student who is not guaranteed an assignment

In [610]:
over_assign

[(6, 2018, 3), (15, 2018, 2), (16, 2018, 1)]

In [611]:
df.loc[(df['School ID'] == 6) & (df['Year'] == 2018) & (df['Grade'] == 3)]

,Student ID,Year,Grade,Choice Rank,School ID,Treatment,Capacity,Priority,Default Tie-breaker Index,Tie-breaker Student Group Index,Advantage,Default Tie-breaker,Effective Tie-breaker,Assignment,Enrollment,Outcome 1,Outcome 2,Covariate 1,Covariate 2,Covariate 3
217,44,2018,3,3,6,1,50,2,3,1,0.3,0.090120,0.063084,0,0,11,67,88,44,154
386,78,2018,3,2,6,1,50,3,3,1,0.3,0.413344,0.289341,0,0,60,22,38,71,190
453,91,2018,3,4,6,1,50,2,10,1,0.3,0.796559,0.557592,0,0,28,84,54,29,424
622,125,2018,3,3,6,1,50,2,19,0,0.0,0.066137,0.066137,0,1,61,54,66,37,403
757,152,2018,3,3,6,1,50,5,9,1,0.3,0.665987,0.466191,0,0,71,91,50,94,79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29254,5851,2018,3,5,6,1,50,4,17,0,0.0,0.846015,0.846015,1,1,38,30,89,100,347
29423,5885,2018,3,4,6,1,50,4,6,1,0.3,0.994395,0.696076,0,0,45,63,91,57,305
29566,5914,2018,3,2,6,1,50,5,8,0,0.0,0.931282,0.931282,0,0,42,16,65,92,145
29923,5985,2018,3,4,6,1,50,2,19,0,0.0,0.874382,0.874382,1,0,27,93,98,80,307


In [612]:
df.loc[(df['School ID'] == 6) & (df['Year'] == 2018) & (df['Grade'] == 3)]['Assignment'].sum()

52

Most of people have non-zero priority.

Thus, Stata should return the 3 schools (6, 15, 16) in the over_assign list.

### (7.2) A (school, year, grade) pair is enrolled by more students than its capacity and contains at least one student who is not guaranteed an assignment

In [613]:
over_enroll

[(16, 2018, 1), (20, 2017, 3)]

Stata should return these two schools

### (8) A student is not assigned to a school although the student was guaranteed an assignment to that school and she was not assigned to any school she prefers to that school.

Student ID 15 \
Assignment \
index 74

### (9) A student is assigned to school s, even though (1) she prefers school s' to s, (2) her applicant position at s' was better than her position at s, (3) there were still available spots at s', (4) and she is eligible at s'

There should be a lot of cases of this. The following case should be included: \
Student ID 16 \
School ID 15 \
while she prefers school 3, 24 and her applicant position is better at those schools. \
Assignment \
index 78

### (10) Abnormally large value (e.g., greater than 10 × mean of that column) found in a column that is unlikely to have a huge outlier

Student ID 18 \
Grade \
index 85-89

### (11) A school uses non-lottery tie-breaker, and correlation between Priority and Tie-breaker within the (school, year, grade) approximates 1

School ID 1 \
Year 2017 \
Grade 1 \
Orange